In [59]:
from laptoppriceworkflow.data import get_data


In [57]:
def get_data():
    """Download and cache the laptop-dataset
    Parameters
    ====
    Returns
    ====
    data : pandas.DataFrame
        The laptop price data
    """
    import pandas as pd
    dataset = pd.read_json("public-dataset.json")
    df = pd.DataFrame(dataset)
    return df

In [60]:
f = get_data()

NameError: name 'pd' is not defined

In [25]:
df.head()

,buynow_price,rodzaj karty graficznej,komunikacja,rozdzielczość (piksele),liczba rdzeni procesora,wielkość pamięci ram,system operacyjny,typ dysku twardego,sterowanie,multimedia,typ pamięci ram,taktowanie bazowe procesora (ghz),seria procesora,pojemność dysku (gb),gwarancja,wielkość matrycy,bluetooth,wi-fi,lan 10/100 mbps,lan 10/100/1000 mbps
70,1599.0,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1366 x 768,2,8 gb,[windows 10 home],hdd,"[klawiatura, touchpad]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr3,1.9,intel core i3,500.0,sprzedawcy,"13"" - 13.9""",1,1,2797,3961
1489,2999.0,grafika dedykowana,"[nfc (near field communication), gps]",1920 x 1080,2,12 gb,[windows 10 home],None,[touchpad],"[czytnik kart pamięci, kamera, głośniki, mikro...",ddr4,2.0,intel core i3,480.0,producenta,"15"" - 15.9""",0,0,2797,3961
407,2999.0,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1366 x 768,2,16 gb,[windows 7 professional 64-bit],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr3l,2.1,intel pentium dual-core,480.0,producenta,"15"" - 15.9""",1,1,2797,3961
1359,2849.0,grafika zintegrowana,"[wi-fi, bluetooth, lan 10/100/1000 mbps]",1920 x 1080,2,8 gb,[windows 10 professional],ssd,"[klawiatura, touchpad, klawiatura numeryczna]","[czytnik kart pamięci, kamera, głośniki, mikro...",ddr4,2.0,intel core i3,480.0,producenta,"15"" - 15.9""",1,1,2797,3961
10567,2319.0,None,None,None,2,None,None,None,None,None,None,NaN,None,NaN,producenta,None,NaN,NaN,2797,3961


In [5]:
df = df.drop("stan", axis = 1) #druga opcja bez zapisu nowego df df.drop("stan", inplace = True)

In [6]:
df.dtypes

buynow_price                         float64
rodzaj karty graficznej               object
komunikacja                           object
rozdzielczość (piksele)               object
liczba rdzeni procesora               object
wielkość pamięci ram                  object
system operacyjny                     object
typ dysku twardego                    object
sterowanie                            object
multimedia                            object
typ pamięci ram                       object
taktowanie bazowe procesora (ghz)    float64
seria procesora                       object
pojemność dysku (gb)                 float64
gwarancja                             object
wielkość matrycy                      object
dtype: object

In [7]:
df["komunikacja"].apply(pd.Series).stack().unique().tolist()

['wi-fi',
 'bluetooth',
 'lan 10/100/1000 mbps',
 'nfc (near field communication)',
 'gps',
 'lan 10/100 mbps',
 'intel wireless display (widi)',
 'modem 3g (wwan)',
 'wi-fi 802.11 a/b/g/n/ac',
 'wi-fi 802.11 b/g/n/ac',
 'modem 4g (lte)',
 'wi-fi 802.11 b/g/n',
 'wi-fi 802.11 a/b/g/n']

In [8]:
df["bluetooth"] = (df["komunikacja"].str.contains('bluetooth', regex=False)*1)

In [9]:
(df["komunikacja"].str.contains('bluetooth', regex=False)*1).sum()

6524

In [10]:
df["wi-fi"] = (df["komunikacja"].str.contains('wi-fi', regex=False)*1)

In [11]:
df["lan 10/100 mbps"] = (df["komunikacja"].str.contains('lan 10/100 mbps', regex=False)*1).sum()

In [12]:
df["lan 10/100/1000 mbps"] = (df["komunikacja"].str.contains('lan 10/100/1000 mbps', regex=False)*1).sum()

In [13]:
(df["komunikacja"].str.contains('nfc (near field communication)', regex=False)*1).sum()

1454

In [14]:
(df["komunikacja"].str.contains('modem 3g (wwan)', regex=False)*1).sum()

1009

In [15]:
(df["komunikacja"].str.contains('gps', regex=False)*1).sum()

269

In [16]:
(df["komunikacja"].str.contains('intel wireless display (widi)', regex=False)*1).sum()

1004

In [17]:
(df["komunikacja"].str.contains('modem 4g (lte)', regex=False)*1).sum()

267

In [18]:
(df["komunikacja"].str.contains('wi-fi 802.11 b/g/n', regex=False)*1).sum()

73

In [19]:
(df["komunikacja"].str.contains('wi-fi 802.11 b/g/n/ac', regex=False)*1).sum()

166

In [20]:
(df["komunikacja"].str.contains('wi-fi 802.11 a/b/g/n/ac', regex=False)*1).sum()

143

In [21]:
(df["komunikacja"].str.contains('wi-fi 802.11 a/b/g/n', regex=False)*1).sum()

17

In [22]:
df.shape

(7853, 20)

In [23]:
160/7853

0.020374379218133197

In [61]:
get_data?